In [2]:

import torch
import sys
sys.path.append('/data/zbw/course/AAAI/project/work/')
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from model.preprocess import MinistDataLoader, TestDataLoader
from torch.utils.data import DataLoader

path = '/data/zbw/course/AAAI/project/work'
batch_size = 64
train_data = MinistDataLoader(root_dir= path  + '/processed_data/train/')
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

val_data = MinistDataLoader(root_dir= path  +'/processed_data/val/')
val_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2)

test_data = TestDataLoader(root_dir= path  +'/processed_data/test/')
test_data_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=2)

class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(10, 6, 5)  
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16*4*4, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [ ]:
lenet5_model = LeNet5()
lenet5_model.train()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on device: {device}")
lenet5_model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(lenet5_model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 2

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = lenet5_model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

lenet5_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in val_data_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = lenet5_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


val_accuracy = 100 * correct / total
print(val_accuracy)

Training on device: cuda:0
[1,   100] loss: 2.306
[1,   200] loss: 2.302
[1,   300] loss: 2.301
[1,   400] loss: 2.297
[1,   500] loss: 2.294
[1,   600] loss: 2.288
[1,   700] loss: 2.276
[1,   800] loss: 2.258
[1,   900] loss: 2.198
[2,   100] loss: 2.043
[2,   200] loss: 1.986
[2,   300] loss: 1.919
[2,   400] loss: 1.779
[2,   500] loss: 1.602
[2,   600] loss: 1.379
[2,   700] loss: 1.249
[2,   800] loss: 1.162
[2,   900] loss: 1.137
torch.Size([64, 10, 28, 28])
64 53
torch.Size([64, 10, 28, 28])
128 104
torch.Size([64, 10, 28, 28])
192 155
torch.Size([64, 10, 28, 28])
256 211
torch.Size([64, 10, 28, 28])
320 259
torch.Size([64, 10, 28, 28])
384 311
torch.Size([64, 10, 28, 28])
448 361
torch.Size([64, 10, 28, 28])
512 417
torch.Size([64, 10, 28, 28])
576 470
torch.Size([64, 10, 28, 28])
640 527
torch.Size([64, 10, 28, 28])
704 580
torch.Size([64, 10, 28, 28])
768 634
torch.Size([64, 10, 28, 28])
832 684
torch.Size([64, 10, 28, 28])
896 727
torch.Size([64, 10, 28, 28])
960 779
torch.

In [9]:
import torchvision.models as models
from torchvision import transforms
class ResizeChannels:
    def __init__(self, size):
        self.size = size

    def __call__(self, img):
        channels, height, width = img.shape
        resized_channels = [F.interpolate(img[c].unsqueeze(0).unsqueeze(0), size=self.size, mode='bilinear', align_corners=False) for c in range(channels)]
        return resized_channels
    

class ModifiedResNet(models.ResNet):
    def __init__(self, block, layers, num_classes=10):
        super(ModifiedResNet, self).__init__(block, layers)
        self.conv1 = nn.Conv2d(10, 64, kernel_size=7, stride=2, padding=3, bias=False)  # 修改为10个通道的输入
        self.fc = nn.Linear(512 * block.expansion, num_classes)



resnet18_model = ModifiedResNet(models.resnet.BasicBlock, [2, 2, 2, 2])

resnet18_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lenet5_model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 1

resnet18_model.train()
for epoch in range(num_epochs):

    running_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        outputs = resnet18_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_data_loader)}")
        
resnet18_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in val_data_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = resnet18_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()


val_accuracy = 100 * correct / total
print(val_accuracy)

Epoch 1/1, Loss: 2.5590361199145124
9.081666666666667
